In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#0) prepare data
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

n_samples, n_features = x.shape
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

#1) model
#f = wx + b, sigmoid at the end

class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

#2) loss and optimizer
criterion = nn.BCELoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#3) training loop
num_epochs = 100
for epoch in range(num_epochs):
  #forward pass and loss
  y_predicted = model(x_train)
  loss = criterion(y_predicted, y_train)
  #backward pass
  loss.backward()
  #updates
  optimizer.step()
  optimizer.zero_grad()
  if (epoch + 1) % 10 == 0:
    print(f"epoch: {epoch + 1}, loss = {loss.item():.4f}")

with torch.no_grad():
  y_predicted = model(x_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f"accuracy = {acc:.4f}")


epoch: 10, loss = 0.4844
epoch: 20, loss = 0.4113
epoch: 30, loss = 0.3628
epoch: 40, loss = 0.3280
epoch: 50, loss = 0.3015
epoch: 60, loss = 0.2807
epoch: 70, loss = 0.2637
epoch: 80, loss = 0.2495
epoch: 90, loss = 0.2374
epoch: 100, loss = 0.2270
accuracy = 0.9035
